In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
# from perception import find_ball
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")
V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
# you can also change the shape & size
targetObj=RealWorld.getFrame("obj0")
targetObj.setColor([1.,0,0])
# targetObj.setShape(ry.ST.sphere, [.03])
RealWorld.getFrame("obj0").setShape(ry.ST.ssBox, [.05, .05, .05, .01])
targetObj.setPosition([-0.1, .3, 0.7])
targetObj.setContact(1)


targetObj1=RealWorld.getFrame("obj1")
targetObj1.setColor([1.,0,0])
# targetObj.setShape(ry.ST.sphere, [.03])
RealWorld.getFrame("obj1").setShape(ry.ST.ssBox, [.05, .05, .05, .01])
targetObj1.setPosition([0, .0, 1.5])
targetObj1.setContact(1)



#remove some objects
for o in range(2,30):
    name = "obj%i" % o
    print("deleting", name)
    RealWorld.delFrame(name)

V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

deleting obj2
deleting obj3
deleting obj4
deleting obj5
deleting obj6
deleting obj7
deleting obj8
deleting obj9
deleting obj10
deleting obj11
deleting obj12
deleting obj13
deleting obj14
deleting obj15
deleting obj16
deleting obj17
deleting obj18
deleting obj19
deleting obj20
deleting obj21
deleting obj22
deleting obj23
deleting obj24
deleting obj25
deleting obj26
deleting obj27
deleting obj28
deleting obj29


In [4]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [5]:
# create your model world
C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
#V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

# save initial background
[rgb0, depth0] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
points0 = S.depthData2pointCloud(depth0, fxfypxpy)

In [6]:
obj = C.addFrame("object")
# obj.setShape(ry.ST.sphere, [.03])
obj.setShape(ry.ST.ssBox, [.05, .05, .05, .01])
obj.setColor([0,1,0])
obj.setContact(1)

p_obj = targetObj.getPosition()   
r_obj = targetObj.getQuaternion()
obj.setPosition(p_obj)
obj.setQuaternion(r_obj)

obj1 = C.addFrame("object1")
# obj.setShape(ry.ST.sphere, [.03])
obj1.setShape(ry.ST.ssBox, [.05, .05, .05, .01])
obj1.setColor([0,1,0])
#obj1.setContact(1)

p_obj = targetObj1.getPosition()   
r_obj = targetObj1.getQuaternion()
obj1.setPosition(p_obj)
obj1.setQuaternion(r_obj)

V.recopyMeshes(C)
V.setConfiguration(C)

In [7]:
points = []
tau = .01
t_object = "object"
closed = 0
for t in range(800):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
       
        # skipping perception
        p_obj = targetObj.getPosition()   
        r_obj = targetObj.getQuaternion()
        #obj.setPosition(p_obj)
        #obj.setQuaternion(r_obj)
        
        
        p_obj = targetObj1.getPosition()   
        r_obj = targetObj1.getQuaternion()
        #obj1.setPosition(p_obj)
        #obj1.setQuaternion(r_obj)

        V.recopyMeshes(C)
        V.setConfiguration(C)
            
    
    if not S.getGripperIsGrasping("R_gripper"):
        [y,J] = C.evalFeature(ry.distance, ["R_gripperCenter", t_object])
        #print(abs(y))
        if abs(y) < 0.01 and closed == 0:
            closed = 1
            print(S.getGripperWidth("R_gripper"))
            S.closeGripper("R_gripper")
            #t_object = "object1"
            print("graspping", t)
            print(S.getGripperWidth("R_gripper"))
            
    else:
        #print("while closing ",S.getGripperWidth("R_gripper"))
        t_object = "object1"
        [y,J] = C.evalFeature(ry.distance, ["R_gripperCenter", t_object])
        if abs(y) < 0.2 and closed == 1:
            S.openGripper("R_gripper")
            print("opening")
            closed = 2
        # add a grasping condition
        # and utilize
        # S.closeGripper("R_gripper")

    # you can add your manipulation code here
    # Tips, not restricted: compute a one-step komo path
    # and compute the configuration q
    # komo = C.komo_path(1.,1,tau,True)

    if closed != 2 :
        komo = C.komo_path(1.,1, tau, True) 
        komo.clearObjectives()
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
        komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
        komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter",t_object], ry.OT.eq, [1e2])
        
        if closed == 0:
            komo.addObjective([1.], ry.FS.scalarProductXZ, [t_object, "R_gripperCenter"], ry.OT.eq, target=[1])
            komo.addObjective([1.], ry.FS.scalarProductYZ, ["R_gripperCenter", t_object], ry.OT.eq)

        komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);
        komo.addObjective([], ry.FS.qItself, ["R_finger1"], ry.OT.eq, [1e1], order=1)
        komo.addObjective([], ry.FS.qItself, ["R_finger2"], ry.OT.eq, [1e1], order=1)
        komo.optimize()
        #komo.getReport()
        C.setFrameState( komo.getConfiguration(0))
        #V.setConfiguration(C)
        q = C.getJointState()  

    S.step(q, tau, ry.ControlMode.position)
    #S.closeGripper()

0.001776045881849055
graspping 70
0.001776045881849055
opening
